In [ ]:
! pip install datasets sentencepiece transformers evaluate rouge_score bert_score huggingface_hub sentencepiece
! pip3 install - U scikit-learn scipy matplotlib


In [1]:
! nvidia-smi

Fri Mar 31 21:24:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:17:00.0 Off |                    0 |
| N/A   33C    P0    37W / 250W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:CA:00.0 Off |                    0 |
| N/A   

In [2]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu
import torch
import torch.nn as nn
from transformers import pipeline


2023-03-31 21:24:53.814245: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 21:24:54.022462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 21:25:00.442865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from datasets import load_dataset

data_files = {"train": "./train.pkl",
              "test": "./test.pkl",
              "val": "./val.pkl"}

raw_dataset = load_dataset("pandas", data_files=data_files, cache_dir=None)

train_data = raw_dataset['train']
test_data = raw_dataset['test']
val_data = raw_dataset['val']


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Dataset pandas downloaded and prepared to /Users/madhavappaneni/.cache/huggingface/datasets/pandas/default-f9f7ecce7864e80a/0.0.0/3ac4ffc4563c796122ef66899b9485a3f1a977553e2d2a8a318c72b8cc6f2202. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(36621, 1)
(5242, 1)
(5712, 1)


In [20]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, GPT2Config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    model_name, model_name, tie_encoder_decoder=False).to(device)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_length = 64
model.config.early_stopping = True
model.config.num_beams = 1
model.config.vocab_size = model.config.encoder.vocab_size

encoder_max_length = 150
decoder_max_length = 150
batch_size = 32


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [21]:
from transformers import DataCollatorForLanguageModeling


def tokenize_function(examples):
    inputs = [ex["input_text"] for ex in examples['conversation']]
    targets = [ex["label"] for ex in examples['conversation']]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=encoder_max_length, truncation=True, padding='max_length'
    )
    return model_inputs


In [22]:
tokenized_datasets = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_dataset["train"].column_names,
)


In [23]:
# ! pip3 install bert_score absl-py rouge_score git+https://github.com/google-research/bleurt.git

In [24]:
import evaluate
rouge = evaluate.load('rouge')
# perplexity = evaluate.load("perplexity", module_type="metric")
# from bleurt import score as bluert_score
# from bleurt import score
# checkpoint = "bleurt/BLEURT-20"
# scorer = score.BleurtScorer(checkpoint)
# scores = scorer.score(references=references, candidates=candidates)
bertscore = evaluate.load("bertscore")


# implement the compute matrix to be used in evaluation the model
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return {"bleu": round(corpus_bleu(pred_str, [label_str]).score, 4),  
            "rouge": rouge.compute(predictions=pred_str, references=label_str),
            "bert": bertscore.compute(predictions=pred_str, references=label_str, lang="en"),
#             'perplexity': perplexity.compute(predictions=pred_str, model_id='bert-base-uncased')
#             "bleurt": scorer.score(predictions=pred_str, references=label_str),
}


In [25]:
# output_path = F"/content/gdrive/My Drive/CC-Train-New"
# output_path = "."

# training_args = TrainingArguments(
#     output_dir=output_path,
#     overwrite_output_dir=True,
#     num_train_epochs=4,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=32,
#     eval_steps=250,
#     warmup_steps=5,
#     save_strategy="no"
#     logging_dir='./logs',
#     learning_rate=2e-5,
# )

# # Initialize trainer and train the model
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['val'],
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForLanguageModeling(
#         tokenizer=tokenizer, mlm=False
#     )
# )


# Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    gradient_accumulation_steps=3,
    save_strategy="no",
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy="epoch",
    do_train=True,
    logging_steps=500,
    save_steps=32965 // (batch_size * 2),
    warmup_steps=100,
    eval_steps=10,
    num_train_epochs=4,
    overwrite_output_dir=True,
    save_total_limit=0,
)


In [26]:
#Set training arguments 
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer
)

In [27]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "{'rouge1': 0.15928207711090248, 'rouge2': 0.0281764929975961, 'rougeL': 0.14513501559928998, 'rougeLsum': 0.1450477155770841}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': [0.8356745839118958, 0.847476601600647, 0.8590662479400635, 0.8887023329734802, 0.8678210973739624, 0.8980492353439331, 0.8731637597084045, 0.8664524555206299, 0.8442089557647705, 0.8966019153594971, 0.8303210735321045, 0.8119657039642334, 0.8732326030731201, 0.8835140466690063, 0.8380638957023621, 0.9166370630264282, 0.8239371180534363, 0.8590268492698669, 0.8710851073265076, 0.8459785580635071, 0.8553431034088135, 0.8339856863021851, 0.8705228567123413, 0.8332104682922363, 0.8577489852905273, 0.8312736749649048, 0.8265132904052734, 0.819130539894104, 0.8475207090377808, 0.8895940184593201, 0.8796237707138062, 0.865

Trainer is attempting to log a value of "{'rouge1': 0.1668286708648823, 'rouge2': 0.036100759738413826, 'rougeL': 0.15422171949004496, 'rougeLsum': 0.15424740969301032}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': [0.8366190791130066, 0.846716582775116, 0.8896841406822205, 0.8370091915130615, 0.8914945125579834, 0.8855212926864624, 0.8497355580329895, 0.8856710195541382, 0.8375992774963379, 0.8926282525062561, 0.8479205369949341, 0.8361694812774658, 0.9024759531021118, 0.8800339698791504, 0.8376517295837402, 0.8955968022346497, 0.8062865734100342, 0.8733381032943726, 0.9333566427230835, 0.8568239808082581, 0.8678553104400635, 0.8686766028404236, 0.8895463943481445, 0.8525350093841553, 0.8577489852905273, 0.8489734530448914, 0.8412203192710876, 0.8493448495864868, 0.8632980585098267, 0.8597558736801147, 0.8709654808044434, 0.

Trainer is attempting to log a value of "{'rouge1': 0.17653796240926478, 'rouge2': 0.04048530519546646, 'rougeL': 0.16204830175047893, 'rougeLsum': 0.16208598717705602}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': [0.8596557378768921, 0.8678449392318726, 0.8726844787597656, 0.8443290591239929, 0.8893115520477295, 0.8855212926864624, 0.8583270907402039, 0.8958274126052856, 0.8656179308891296, 0.8844535946846008, 0.8603589534759521, 0.832874059677124, 0.8864823579788208, 0.8721923828125, 0.8659409284591675, 0.9031842947006226, 0.8195546865463257, 0.8671706914901733, 0.9169243574142456, 0.8506240844726562, 0.8929930329322815, 0.8728041648864746, 0.8872548341751099, 0.8503249883651733, 0.8806309700012207, 0.87950199842453, 0.8395285606384277, 0.8333540558815002, 0.8498919010162354, 0.8833531737327576, 0.8929730653762817, 0.84235

Trainer is attempting to log a value of "{'rouge1': 0.17329611105538248, 'rouge2': 0.03930270493670318, 'rougeL': 0.15975779949078994, 'rougeLsum': 0.15966065718593497}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': [0.8403416872024536, 0.8577944040298462, 0.8471662998199463, 0.8443290591239929, 0.889566957950592, 0.8855212926864624, 0.8483332395553589, 0.8958274722099304, 0.8431912660598755, 0.8908639550209045, 0.8677448630332947, 0.8328740000724792, 0.8960786461830139, 0.8910533785820007, 0.82574862241745, 0.8985419273376465, 0.8434398770332336, 0.8660219311714172, 0.9091076254844666, 0.8506240844726562, 0.8619425296783447, 0.871071457862854, 0.8895463943481445, 0.8340014219284058, 0.8577489852905273, 0.8460446000099182, 0.8241750597953796, 0.8298397064208984, 0.8498919606208801, 0.8831232190132141, 0.8882031440734863, 0.848

TrainOutput(global_step=764, training_loss=0.7209476450975029, metrics={'train_runtime': 2284.6376, 'train_samples_per_second': 64.117, 'train_steps_per_second': 0.334, 'total_flos': 2.6326526500404e+16, 'train_loss': 0.7209476450975029, 'epoch': 4.0})

In [28]:
# save the model
trainer._save("./trained_model/model")
tokenizer.save_pretrained("./trained_model/tokenizer")

('./trained_model/tokenizer/tokenizer_config.json',
 './trained_model/tokenizer/special_tokens_map.json',
 './trained_model/tokenizer/vocab.txt',
 './trained_model/tokenizer/added_tokens.json',
 './trained_model/tokenizer/tokenizer.json')

In [29]:

results = trainer.evaluate()

Trainer is attempting to log a value of "{'rouge1': 0.17329611105538248, 'rouge2': 0.03930270493670318, 'rougeL': 0.15975779949078994, 'rougeLsum': 0.15966065718593497}" of type <class 'dict'> for key "eval/rouge" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': [0.8403416872024536, 0.8577944040298462, 0.8471662998199463, 0.8443290591239929, 0.889566957950592, 0.8855212926864624, 0.8483332395553589, 0.8958274722099304, 0.8431912660598755, 0.8908639550209045, 0.8677448630332947, 0.8328740000724792, 0.8960786461830139, 0.8910533785820007, 0.82574862241745, 0.8985419273376465, 0.8434398770332336, 0.8660219311714172, 0.9091076254844666, 0.8506240844726562, 0.8619425296783447, 0.871071457862854, 0.8895463943481445, 0.8340014219284058, 0.8577489852905273, 0.8460446000099182, 0.8241750597953796, 0.8298397064208984, 0.8498919606208801, 0.8831232190132141, 0.8882031440734863, 0.848

In [30]:
results

{'eval_loss': 0.3268454670906067,
 'eval_bleu': 2.1963,
 'eval_rouge': {'rouge1': 0.17329611105538248,
  'rouge2': 0.03930270493670318,
  'rougeL': 0.15975779949078994,
  'rougeLsum': 0.15966065718593497},
 'eval_bert': {'precision': [0.8403416872024536,
   0.8577944040298462,
   0.8471662998199463,
   0.8443290591239929,
   0.889566957950592,
   0.8855212926864624,
   0.8483332395553589,
   0.8958274722099304,
   0.8431912660598755,
   0.8908639550209045,
   0.8677448630332947,
   0.8328740000724792,
   0.8960786461830139,
   0.8910533785820007,
   0.82574862241745,
   0.8985419273376465,
   0.8434398770332336,
   0.8660219311714172,
   0.9091076254844666,
   0.8506240844726562,
   0.8619425296783447,
   0.871071457862854,
   0.8895463943481445,
   0.8340014219284058,
   0.8577489852905273,
   0.8460446000099182,
   0.8241750597953796,
   0.8298397064208984,
   0.8498919606208801,
   0.8831232190132141,
   0.8882031440734863,
   0.848946750164032,
   0.8880815505981445,
   0.878561139

In [33]:
import pickle

exampleObj = results

fileObj = open('data.obj', 'wb')
pickle.dump(exampleObj,fileObj)
fileObj.close()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("ED-BERT2BERT")
tokenizer.push_to_hub("ED-BERT2BERT")
